# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298353485114                   -0.85    5.2         
  2   -8.300198651455       -2.73       -1.26    1.4    128ms
  3   -8.300434593798       -3.63       -1.88    1.2    122ms
  4   -8.300461349229       -4.57       -2.65    4.1    197ms
  5   -8.300464502300       -5.50       -3.11    2.4    139ms
  6   -8.300464608232       -6.97       -3.30    1.8    119ms
  7   -8.300464625412       -7.76       -3.53    1.1    109ms
  8   -8.300464634369       -8.05       -3.70   13.5    321ms
  9   -8.300464638674       -8.37       -3.82    1.2    113ms
 10   -8.300464642560       -8.41       -4.05    2.0    127ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67454505584                   -0.70    6.5         
  2   -16.67871728078       -2.38       -1.14    1.0    314ms
  3   -16.67920874818       -3.31       -1.87    2.5    344ms
  4   -16.67927635288       -4.17       -2.77    2.2    343ms
  5   -16.67928596622       -5.02       -3.17    7.9    540ms
  6   -16.67928619855       -6.63       -3.46    2.4    331ms
  7   -16.67928621807       -7.71       -3.94    1.8    282ms
  8   -16.67928622144       -8.47       -4.56    2.5    330ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32710512485                   -0.56    7.9         
  2   -33.33444232651       -2.13       -1.00    1.2    1.08s
  3   -33.33597054949       -2.82       -1.72    3.9    1.34s
  4   -33.33618465556       -3.67       -2.62    3.9    1.40s
  5   -33.33694136474       -3.12       -2.94    9.4    1.90s
  6   -33.33694331174       -5.71       -3.42    3.0    1.16s
  7   -33.33694376867       -6.34       -4.26    3.2    1.36s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298441951623                   -0.85    5.5         
  2   -8.300265438746       -2.74       -1.59    1.0   90.0ms
  3   -8.300437916098       -3.76       -2.73    2.4    111ms
  4   -8.300458782523       -4.68       -2.85    5.2    188ms
  5   -8.300464070519       -5.28       -3.27    2.4    121ms
  6   -8.300464596836       -6.28       -3.75    1.6    118ms
  7   -8.300464642816       -7.34       -4.54    2.2    152ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32694205560                   -0.56    6.6         
  2   -33.30498110331   +   -1.66       -1.26    2.1    984ms
  3   -3.275634532702   +    1.48       -0.33    6.2    2.11s
  4   -33.33549083863        1.48       -2.01    5.4    1.84s
  5   -33.28281405120   +   -1.28       -1.52    4.5    1.67s
  6   -33.01135113671   +   -0.57       -1.30    4.5    1.59s
  7   -33.26193504919       -0.60       -1.58    5.1    1.58s
  8   -33.33561089750       -1.13       -2.39    3.6    1.34s
  9   -33.33522292307   +   -3.41       -2.38    2.1    1.09s
 10   -33.33671365692       -2.83       -2.59    2.5    1.14s
 11   -33.33692995564       -3.66       -3.18    2.0    1.05s
 12   -33.33694127237       -4.95      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.